# Actividad 4
## Alumno
Noé Guerrero Ascencio


# Módulos

In [1]:
import panel as pn
import panel.widgets as pnw
#pn.extension("plotly")
import holoviews as hv

pn.extension(sizing_mode = 'stretch_width')

import math
import numpy as np
import pandas as pd

import plotly.graph_objects as go
from scipy.stats import wrapcauchy
from scipy.stats import levy_stable

# Clases

In [118]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:            
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y
            
    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)
    
    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)
    
    #Set step length
    def setlength(self, value):
        length = self.get_length()
        self.x *= value/length
        self.y *= value/length
    
    # rotate vector
    def rotated(self, angle):        
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)
    
class DashBoard:
    trayectoria_actual = pd.DataFrame()

# Funciones

## Funciones de recorridos aleatorios

In [97]:
def bm_2d(n_steps = 1000, speed = 6, s_pos=[0, 0]):
    """
      Arguments:
        n_steps:
        speed:
        s_pos:
      Returns:
        BM_2d_df
    """
    #n_steps = 1000
    #s_pos = [0,0]
    #speed = 6

    # Init velocity vector
    velocity = Vec2d(speed, 0)

    BM_2d_df = pd.DataFrame(columns = ["x_pos", "y_pos"])
    temp_df = pd.DataFrame([{"x_pos": s_pos[0], "y_pos": s_pos[1]}])

    BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

    for i in range(n_steps - 1):
        turn_angle = np.random.uniform(low=-np.pi, high=np.pi)
        velocity = velocity.rotated(turn_angle)

        temp_df = pd.DataFrame([{"x_pos": BM_2d_df.x_pos[i] + velocity.x, "y_pos": BM_2d_df.y_pos[i] + velocity.y}])
        BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)
    #return DF
    return BM_2d_df

def CRW_2d(n_steps = 1000, speed = 6, coef = 0.6, s_pos=[2, 5]):
    """
      Arguments:
        n_steps:
        speed:
        coef: El coeficiente de wrapcauchy con el que se desea
        s_pos:
      Returns:
        BM_2d_df
    """
    #n_steps = 1000
    #s_pos = [0,0]
    #speed = 6

    # Init velocity vector
    velocity = Vec2d(speed, 0)

    BM_2d_df = pd.DataFrame(columns = ["x_pos", "y_pos"])
    temp_df = pd.DataFrame([{"x_pos": s_pos[0], "y_pos": s_pos[1]}])

    BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

    for i in range(n_steps - 1):
        turn_angle = wrapcauchy.rvs(c=coef)
        #print(turn_angle)
        #turn_angle = np.random.uniform(low=-np.pi, high=np.pi)
        velocity = velocity.rotated(turn_angle)

        temp_df = pd.DataFrame([{"x_pos": BM_2d_df.x_pos[i] + velocity.x, "y_pos": BM_2d_df.y_pos[i] + velocity.y}])
        BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)
    #return DF
    return BM_2d_df

def Levy_2d(n_steps = 1000, alpha = 1.9, wrapcauchy_exponent = 0.7, speed=3.0, s_pos = [0, 0]):
    

    # Variables para Levy
    Levy_exponent = alpha #Exponente de Levy
    std_motion_steps = speed #Número de pasos promedio que dara nuestro agente
    beta_exp = 0 #Se usa beta = 0 y la gráfica toma una figura de campana de Gauss

    # Init velocity vector
    speed = std_motion_steps
    velocity_vector = Vec2d(speed, 0)

    #times aux
    time = np.linspace(0, 1, n_steps)

    # Init dataframe
    LW_3d_df = pd.DataFrame(columns=["x_pos", "y_pos"])

    # auxiliar para concatenar
    temp_df = pd.DataFrame([{"x_pos":s_pos[0], "y_pos": s_pos[1]}])
    LW_3d_df = pd.concat([LW_3d_df, temp_df], ignore_index=True)

    for i in range(1, n_steps):
        #print(speed)
        turn_angle = wrapcauchy.rvs(wrapcauchy_exponent) #Giramos con la distribución wrapcauchy
        new_speed = levy_stable.rvs(alpha=Levy_exponent, beta=beta_exp, loc=std_motion_steps)
        #Se establece la nueva velocidad o step length
        velocity_vector.setlength(new_speed)
        
        #Luego giramos
        velocity_vector = velocity_vector.rotated(turn_angle)
        
        temp_df = pd.DataFrame([{"x_pos": LW_3d_df.x_pos[i - 1] + velocity_vector.x, "y_pos": LW_3d_df.y_pos[i - 1] + velocity_vector.y}])
        LW_3d_df = pd.concat([LW_3d_df, temp_df], ignore_index=True)

        #Actualizar speed
        speed = new_speed
    
    return LW_3d_df
    
"""def Levy_2d(alpha=1.0, beta=1.0, loc=3.0, n_samples=100000):
    # Init params
    speed = 1

    s_pos = {"x": 0, "y": 0}

    #Init velocity vector
    velocity = Vec2d(speed, 0)

    # Init DataFrame
    LW_2d_df = pd.DataFrame([{"x_pos": s_pos["x"], "y_pos": s_pos["y"]}])

    i = 1
    while i < n_samples:
        # Get random n_steps
        step_size = levy_stable.rvs(alpha, beta, loc)
        step_size = int(np.ceil(abs(step_size)))

        theta = wrapcauchy.rvs(c=0.7, loc=0)

        velocity = velocity.rotated(theta)

        for j in range(step_size):
            temp_df = pd.DataFrame([{
              "x_pos": LW_2d_df.x_pos[i - 1] + velocity.x,
              "y_pos": LW_2d_df.y_pos[i - 1] + velocity.y
            }])

        # Add to the end to Levy's DF
        LW_2d_df = pd.concat([LW_2d_df, temp_df], ignore_index=True)
        i += 1
    return LW_2d_df"""

def generate_Levy_pdf(loc = 3.0, beta = 1.0, alpha=0.7, resolution = 400):    
    aux_domain = np.linspace(0, 30, resolution)
    Levy_pdf = np.array([levy_stable.pdf(i, alpha, beta, loc) for i in aux_domain])
    return Levy_pdf


## Funciones adicionales

In [81]:
def get_angle(v1, v2, centro):
    #https://muthu.co/using-the-law-of-cosines-and-vector-dot-product-formula-to-find-the-angle-between-three-points/
    BA = [centro[0] - v1[0], centro[1] - v1[1]]
    BC = [v2[0] - centro[0], v2[1] - centro[1]]
    coseno_angulo = np.dot(BA, BC) / (np.linalg.norm(BA) * np.linalg.norm(BC))

    #Se normalizan los decimales (máximo 7)
    coseno_angulo = round(coseno_angulo, 7)
    angulo = np.arccos(coseno_angulo)

    producto_cruz = (BA[0] * BC[1]) - (BA[1] * BC[0])

    #Se normaliza el 0
    diferencia = np.abs(0 - angulo)
    if 0.0001 > diferencia: #Considerarlo 0
      angulo = 0
    return angulo if producto_cruz >= 0 else angulo * -1

def get_distance(vector_1, vector_2):
    """
      Gets the distance between to polar coordenates given by 2 arrays or lists
      Arguments:
        vector_1: A list or array whose first element (position 0) represents the 'x' coordenate and the second (position 1) 'y' 
        vector_2: A list or array whose first element (position 0) represents the 'x' coordenate and the second (position 1) 'y'
      Returns;
        The listance between the given coordinates
    """
    return math.sqrt((vector_1[0] - vector_2[0])**2 + (vector_1[1] - vector_2[1])**2)

In [5]:
Integer_A = pnw.IntInput(name="An Integer Value", value=33, step=1, start=0, end=200)
Integer_A

IntInput(end=200, name='An Integer Value', sizing_mode='stretch_width', start=0, value=33)

In [ ]:
# Load existing trajectories
# BM speed = 6
BM_2d_df_6 = pd.read_csv('trajectories/brownian_6.csv')

# Load existing trajectories
# CRW speed = 6, c = 0.9
CRW_2d_df_9 = pd.read_csv('trajectories/crw_6_9.csv')


# Empty MSD_BM
MSD_BM = np.empty(shape=(0))
total = BM_2d_df_6.shape[0]

# MSD for BM_2d_df_6
for tau in range(1,BM_2d_df_6.shape[0]):
    ## start - Add your code here
    MSD_tau = np.empty(shape=(0))
    for n in range(0, total - tau, 1):
      vector_n = [BM_2d_df_6.iloc[n].x_pos, BM_2d_df_6.iloc[n].y_pos]
      vector_tau_mas_n = [BM_2d_df_6.iloc[tau + n].x_pos, BM_2d_df_6.iloc[tau + n].y_pos]
      MSD_tau = np.append(MSD_tau, (get_distance(vector_n, vector_tau_mas_n)**2))
    #print(len(MSD_tau))
    MSD_BM = np.append(MSD_BM, np.sum(MSD_tau)/len(MSD_tau))
    ## end - Add your code here

# Empty MSD_CRW
MSD_CRW = np.empty(shape=(0))
total = CRW_2d_df_9.shape[0]
# MSD for CRW_2d_df_9

for tau in range(1,CRW_2d_df_9.shape[0]):
    MSD_tau = np.empty(shape=(0))
    for n in range(0, total - tau, 1):
      vector_n = [CRW_2d_df_9.iloc[n].x_pos, CRW_2d_df_9.iloc[n].y_pos]
      vector_tau_mas_n = [CRW_2d_df_9.iloc[tau + n].x_pos, CRW_2d_df_9.iloc[tau + n].y_pos]
      MSD_tau = np.append(MSD_tau, (get_distance(vector_n, vector_tau_mas_n)**2))

    MSD_CRW = np.append(MSD_CRW, np.sum(MSD_tau)/(total - tau))

# Save metrics to Dataframe
## start - Add your code here
met_df_2 = pd.DataFrame(MSD_BM, columns=["MSD_BM"])
met_df_2["MSD_CRW"] = MSD_CRW
## end - Add your code here

# Write to csv
## start - Add your code here
met_df_2.to_csv("trajectories/actividad_2_MSD.csv", index=False)
## end - Add your code here

# Init figure
fig_MSD = go.Figure()

# first trace MSD_BM
fig_MSD.add_trace(go.Scatter(x = np.arange(len(MSD_BM)),
                              y = MSD_BM,
                              marker = dict(size=2),
                              line = dict(width=2),
                              mode = 'lines',
                              name = "MSD BM 6",
                              showlegend = True))

fig_MSD.add_trace(go.Scatter(x = np.arange(len(MSD_CRW)),
                              y = MSD_CRW,
                              marker = dict(size=2),
                              line = dict(width=2),
                              mode = 'lines',
                              name = "MSD CRW 6 c=0.9",
                              showlegend = True))

fig_MSD.show()

In [8]:
css = '''
.centrado{
    text-align: center !important;
}
.bk.elemento-centrado{
    /* Esta clase requiere que el elemento tenga width definido */
    left: 0;
    right: 0;
    margin-left: auto;
    margin-right: auto;
    
}
.bk.botones_tipo_recorrido {
    width: 300px;
    margin: 5px auto;
    text-align: center;
}
'''
pn.extension(raw_css=[css])

In [106]:
# Elementos
radio_group_caminata = pnw.RadioButtonGroup(
    name='Tipo Caminata', options=['BM', 'CRW', 'LF'], button_type='success')
radio_group_metrica = pnw.RadioButtonGroup(
    name='Metrica', options=['MSD', 'Path length'], button_type='success')

number_of_steps = pnw.IntSlider(name='Number of steps', start=1, end=1000, step=1, value=300,
                             sizing_mode='fixed', width=350, css_classes=['elemento-centrado'])

start_x = pnw.IntInput(name="x starting pos", value=15, step=1, start=0, end=50, sizing_mode='fixed',
                      width=250)

start_y = pnw.IntInput(name="y starting pos", value=15, step=1, start=0, end=50, sizing_mode='fixed',
                      width=250)

speed = pnw.IntSlider(name='speed', start=1, end=10, step=1, value=3,
                             sizing_mode='fixed', width=350, css_classes=['elemento-centrado'])
#Coeficiente para Wrapcouchy
wrapcauchy_coef = pnw.FloatSlider(name="wrapcauchy coeficient", value=0.6, step=0.1, start=0.1, end=.9,
                                 sizing_mode='fixed', width=350, height=40, css_classes=['elemento-centrado'])

fila_wrapcauchy_coef = pn.Row(wrapcauchy_coef, visible=False, css_classes=['elemento-centrado'])

# Coeficiente para Levy
levy_coef = pnw.FloatSlider(name="Levy alpha coeficient", value=1.2, step=0.1, start=0.1, end=1.9,
                                 sizing_mode='fixed', width=350, height=40, css_classes=['elemento-centrado'])

fila_levy_coef = pn.Row(levy_coef, visible=False, css_classes=['elemento-centrado'])


In [119]:
#Funciones
def graficar_metrica():
    

@pn.depends(radio_group_caminata, start_x, start_y, speed, number_of_steps, wrapcauchy_coef, levy_coef)
def graficar_trayectoria(radio_group_caminata, start_x, start_y, speed, number_of_steps, wrapcauchy_coef, levy_coef):
    # Obtener parametros
    tipo_trajectoria = radio_group_caminata
    starting_position = [start_x, start_y]
    #n_steps = number_of_steps.value
    #speed_val = speed.value
    
    # Get trajectory (must be a DataFrame with x_pos and y_pos)
    if tipo_trajectoria == "BM":
        title = f"Brownian Motion {number_of_steps} steps"
        line = f"BM, {number_of_steps} steps;"
        trajectory = bm_2d(n_steps=number_of_steps, speed=speed, s_pos=starting_position)
        fila_wrapcauchy_coef.visible = False
        fila_levy_coef.visible = False
    elif tipo_trajectoria == "CRW":
        title = f"Correlated Random Walk {number_of_steps} steps"
        line = f"CRW, coef={round(wrapcauchy_coef,2)};"
        trajectory = CRW_2d(n_steps = number_of_steps, speed = speed, coef = wrapcauchy_coef, s_pos=starting_position)
        fila_wrapcauchy_coef.visible = True
        fila_levy_coef.visible = False
    elif tipo_trajectoria == "LF":
        title = f"Levy Flight {number_of_steps} steps"
        line = f"LF, coef={round(wrapcauchy_coef, 2)}; alpha={round(levy_coef, 2)};"
        trajectory = Levy_2d(n_steps = number_of_steps, alpha = levy_coef, wrapcauchy_exponent = wrapcauchy_coef,
                             speed=3.0, s_pos = starting_position)
        fila_wrapcauchy_coef.visible = True
        fila_levy_coef.visible = True
        
    # Se asigna a la variable de la clase
    DashBoard.trayectoria_actual = trajectory
    
    # Se crea la gráfica
    fig_trajectory_rw = go.Figure()
    fig_trajectory_rw.update_layout(
        title_text = title,
        height = 600,
        scene = dict(
            xaxis = dict(title = "x pos"),
            yaxis = dict(title = "y pos"),
            zaxis = dict(title = "time")
        )
    )
    fig_trajectory_rw.add_trace(
        go.Scatter3d(
            x = trajectory.x_pos,
            y = trajectory.y_pos,
            z = trajectory.index,
            marker = {"size": 2},
            line = {"color": "red", "width": 2},
            mode = "lines",
            name = line,
            showlegend = True
        )
    )
    
    return fig_trajectory_rw



In [123]:
DashBoard.trayectoria_actual

,x_pos,y_pos
0,1,15
1,-4.396168,17.623237
2,-1.058789,22.609407
3,3.258452,26.776113
4,6.670365,31.711581
...,...,...
305,148.762006,194.841482
306,143.903253,198.361782
307,143.158721,192.408155
308,145.004746,198.117112


In [121]:
bootstrap = pn.template.BootstrapTemplate(title='My Dashboard - Tópico de Industria I')
bootstrap.main.append(
    pn.Column(
        "# Elija una caminata",
        pn.Row(
            pn.Spacer(),
            radio_group_caminata,
            pn.Spacer()
        ),
        pn.Row(
            pn.Column(
                "## Controles",
                number_of_steps,
                pn.Row(
                    start_x,
                    start_y,
                    css_classes=['elemento-centrado'],
                ),
                speed,
                fila_wrapcauchy_coef,
                fila_levy_coef
            ),
            pn.Column(
                "## 3D Graph",
                graficar_trayectoria
            )
        ),
        css_classes=['centrado'],
        
    )
)
bootstrap.main.append(
    pn.Column(
        "# Elija una métrica",
        pn.Row(
            pn.Spacer(),
            radio_group_metrica,
            pn.Spacer()
        ),
        # Gráfica
        css_classes=['centrado']
    )
    
)



bootstrap.show()

Launching server at http://localhost:60901


In [57]:
bootstrap = pn.template.BootstrapTemplate(title='My Dashboard - Tópico de Industria I')

xs = np.linspace(0, np.pi)
freq = pn.widgets.FloatSlider(name="Frequency", start=0, end=10, value=2)
phase = pn.widgets.FloatSlider(name="Phase", start=0, end=np.pi)

@pn.depends(freq=freq, phase=phase)
def sine(freq, phase):
    return hv.Curve((xs, np.sin(xs*freq+phase))).opts(
        responsive=True, min_height=400)

@pn.depends(freq=freq, phase=phase)
def cosine(freq, phase):
    return hv.Curve((xs, np.cos(xs*freq+phase))).opts(
        responsive=True, min_height=400)

bootstrap.sidebar.append(freq)
bootstrap.sidebar.append(phase)

bootstrap.main.append(
    pn.Row(
        pn.Card(hv.DynamicMap(sine), title='Sine'),
        pn.Card(hv.DynamicMap(cosine), title='Cosine')
    )
)

In [58]:
bootstrap.show()

Launching server at http://localhost:55383
